# EOPF S3 OLCI L1 Product Data Structure Proposal


In [1]:
import os
import xarray as xr
import glob
from utils import display, get_ds_variables as get_ds
from EOProductDataStructure import EOProductBuilder, EOVariableBuilder, EOGroupBuilder


In [2]:
path_to_product = glob.glob("data/S3A_OL_1*.SEN3")[0]

# Groups definition
groups = {}
groups['coordinates'] = get_ds(path_to_product, ["time_coordinates","geo_coordinates"])
groups['tiepoints'] = get_ds(path_to_product, ["tie_geo_coordinates"])
groups['measurements'] = get_ds(path_to_product,["Oa%02d_radiance" %r for r in range(21)]+["qualityFlags"])
groups['geometry'] = get_ds(path_to_product,["tie_geometries"])
groups['meteo'] = get_ds(path_to_product,["tie_meteo"])
groups['instrument'] = get_ds(path_to_product,["instrument_data"])
groups['orphans'] = get_ds(path_to_product,["removed_pixels"])

# Create a new EOProduct instance
product_name = os.path.basename("S3_OLCI_L1")
product = EOProductBuilder("S3_OLCI_L1", coords=EOGroupBuilder('coordinates'))
# do the same work as before 
product.metadatas = ["xfdumanifest.xml"]

# global attributes
#get them from a netcdf file 
ds = xr.open_dataset(path_to_product + "/Oa01_radiance.nc",decode_times=False,mask_and_scale=False)
product.attrs = {att: f'{value[:50]} ...' if isinstance(value, str) else value for att, value in ds.attrs.items()}
    
# ==================== Product groups setting ========================

for group_name, ds in groups.items():
    group = EOGroupBuilder(group_name)
    group.attrs['description'] = f"{group_name} Data Group"
    group.dims = ds.dims

    for v in ds.variables:
        variable = EOVariableBuilder(v, default_attrs = False)
        var = ds.get(v)
        variable.dtype = var.dtype
        variable.dims = var.dims
        variable.attrs = var.attrs
        group.variables.append(variable)

    product.groups.append(group)

product.attrs['metadata_files'] = '[xfdumanfist.xml]'

In [3]:
display(product.compute())